In [5]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#Initialize PySpark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BigDataAnalysis") \
    .getOrCreate()


In [7]:
#Load Large Datasets
df = spark.read.csv(r"C:\Users\pcm\OneDrive\Desktop\CODTECH Internship\bigdata_employees.csv", header=True, inferSchema=True)
df.show(5)

+----------+------+----------+------+----------+
|EmployeeID|  Name|Department|Salary|Experience|
+----------+------+----------+------+----------+
|         1| Grace|   Finance| 89401|        22|
|         2| David|   Finance| 84602|        29|
|         3|Hannah|        HR| 32814|        17|
|         4|  Emma| Marketing| 81207|        21|
|         5| Grace| Marketing| 32308|        13|
+----------+------+----------+------+----------+
only showing top 5 rows



In [8]:
#Data Exploration
#Check the schema
df.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [9]:
#Check data statistics
df.describe().show()

+-------+-----------------+------+----------+-----------------+-----------------+
|summary|       EmployeeID|  Name|Department|           Salary|       Experience|
+-------+-----------------+------+----------+-----------------+-----------------+
|  count|           100000|100000|    100000|           100000|           100000|
|   mean|          50000.5|  NULL|      NULL|      75126.43523|         14.98501|
| stddev|28867.65779668774|  NULL|      NULL|25913.91447592236|8.379301129973472|
|    min|                1| Alice|   Finance|            30000|                1|
|    max|           100000|  Jack|     Sales|           119999|               29|
+-------+-----------------+------+----------+-----------------+-----------------+



In [10]:
#Count records
df.count()

100000

In [11]:
#Data Cleaning
#Handle missing values
df = df.dropna()  # Drop rows with null values

In [12]:
#Remove duplicates
df = df.dropDuplicates()

In [13]:
#Filter data
df_filtered = df.filter(df["salary"] > 50000)

In [14]:
#Data Transformation
#Convert column types
from pyspark.sql.functions import col

df = df.withColumn("salary", col("salary").cast("double"))


In [15]:
#Create new columns
from pyspark.sql.functions import when

df = df.withColumn("salary_category", when(df.salary > 60000, "High").otherwise("Low"))

In [16]:
#Aggregations
#Group by and count:
df.groupBy("department").count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|20066|
|        HR|19852|
|   Finance|19873|
| Marketing|19991|
|        IT|20218|
+----------+-----+



In [17]:
#Find the average salary per department
df.groupBy("department").avg("salary").show()

+----------+-----------------+
|department|      avg(salary)|
+----------+-----------------+
|     Sales|75352.31067477325|
|        HR|74966.79105379811|
|   Finance|75102.55220651135|
| Marketing|75269.87639437747|
|        IT|74940.65693936097|
+----------+-----------------+



In [18]:
#SQL Queries
#Convert PySpark DataFrame to a SQL table
df.createOrReplaceTempView("employees")
spark.sql("SELECT department, AVG(salary) FROM employees GROUP BY department").show()


+----------+-----------------+
|department|      avg(salary)|
+----------+-----------------+
|     Sales|75352.31067477325|
|        HR|74966.79105379811|
|   Finance|75102.55220651135|
| Marketing|75269.87639437747|
|        IT|74940.65693936097|
+----------+-----------------+



In [30]:
#Use cache() or persist() for frequent queries
df.cache()
df.show()

+----------+-------+----------+--------+----------+---------------+
|EmployeeID|   Name|Department|  salary|Experience|salary_category|
+----------+-------+----------+--------+----------+---------------+
|        13| Hannah|     Sales| 79460.0|        28|           High|
|       152|  Grace|        IT|110739.0|        14|           High|
|       259|  Isaac|        IT|113141.0|         7|           High|
|       500|Charlie|        IT| 92402.0|        29|           High|
|       512|  Isaac| Marketing| 95750.0|        19|           High|
|       549|   Emma|        HR|110472.0|         6|           High|
|       599|  Alice|        IT|115290.0|         3|           High|
|       689|Charlie| Marketing| 46918.0|        16|            Low|
|      1363| Hannah|   Finance| 91911.0|         7|           High|
|      1734|  David|     Sales| 66205.0|         8|           High|
|      2244|  Alice|        HR| 39136.0|         1|            Low|
|      2611|   Jack|   Finance|119725.0|        

In [32]:
#Machine Learning with PySpark
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Prepare features
assembler = VectorAssembler(inputCols=["Experience", "salary"], outputCol="features")
df_ml = assembler.transform(df)

# Train a model
lr = LinearRegression(featuresCol="features", labelCol="salary")
model = lr.fit(df_ml)

# Predict
predictions = model.transform(df_ml)
predictions.select("features", "salary", "prediction").show()


+---------------+--------+------------------+
|       features|  salary|        prediction|
+---------------+--------+------------------+
| [28.0,79460.0]| 79460.0| 79459.99999999999|
|[14.0,110739.0]|110739.0|110738.99999999997|
| [7.0,113141.0]|113141.0|113140.99999999997|
| [29.0,92402.0]| 92402.0| 92401.99999999997|
| [19.0,95750.0]| 95750.0| 95749.99999999997|
| [6.0,110472.0]|110472.0|110471.99999999997|
| [3.0,115290.0]|115290.0|115289.99999999997|
| [16.0,46918.0]| 46918.0| 46918.00000000001|
|  [7.0,91911.0]| 91911.0| 91910.99999999997|
|  [8.0,66205.0]| 66205.0| 66204.99999999999|
|  [1.0,39136.0]| 39136.0|39136.000000000015|
|[14.0,119725.0]|119725.0|119724.99999999996|
|  [7.0,81854.0]| 81854.0| 81853.99999999999|
|[28.0,117831.0]|117831.0|117830.99999999997|
|[12.0,119650.0]|119650.0|119649.99999999996|
| [24.0,71826.0]| 71826.0| 71825.99999999999|
|  [2.0,31100.0]| 31100.0| 31100.00000000002|
|  [1.0,79150.0]| 79150.0| 79149.99999999999|
|  [4.0,61108.0]| 61108.0|        